# API calls

In [ ]:
import requests
import json

# Configurar los endpoints y la URL base
BASE_URL = "https://zara-boost-hackathon.nuwe.io"  # Cambia esto por la URL de tu API
USERS_ENDPOINT = "/users"

# Ruta del archivo donde se guardará la información
OUTPUT_FILE = "users_data.json"

def get_all_users(base_url):
    """Obtiene todos los usuarios del endpoint /users."""
    response = requests.get(f"{base_url}{USERS_ENDPOINT}")
    response.raise_for_status()  # Lanza un error si el request falla
    return response.json()  # Devuelve la lista de usuarios


In [2]:
users = get_all_users(BASE_URL)

In [ ]:
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

OUTPUT_FILE = "users_data.json"

def get_user_by_id(base_url, user_id):
    """Función para realizar una solicitud GET al endpoint del usuario."""
    response = requests.get(f"{base_url}{USERS_ENDPOINT}/{user_id}")
    response.raise_for_status()
    return response.json()

def fetch_all_users_parallel(base_url, users, max_workers=14):
    """Función para obtener información de usuarios en paralelo."""
    users_data = []
    total_users = len(users)

    # Usar ThreadPoolExecutor para paralelizar las solicitudes
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Enviar tareas al pool de hilos
        future_to_user = {
            executor.submit(get_user_by_id, base_url, user_id): user_id
            for user_id in users
        }
        
        for idx, future in enumerate(as_completed(future_to_user), start=1):
            user_id = future_to_user[future]
            try:
                user_data = future.result()  # Obtener el resultado de la tarea
                users_data.append(user_data)
                # Mostrar progreso
                print(
                    f"Processed {idx}/{total_users} users ({(idx / total_users) * 100:.2f}%)",
                    end="\r",
                )
            except Exception as e:
                print(f"\nError processing user {user_id}: {e}")

    # Imprimir un salto de línea al terminar para evitar sobrescribir el mensaje final
    print()
    return users_data


# Obtener los datos de los usuarios en paralelo
users_data = fetch_all_users_parallel(BASE_URL, users, max_workers=20)

# Guardar los datos en un archivo JSON
with open(OUTPUT_FILE, "w") as json_file:
    json.dump(users_data, json_file, indent=4)

print(f"Datos de usuarios guardados en {OUTPUT_FILE}")


Processed 557006/557006 users (100.00%)
Datos de usuarios guardados en users_data.json


In [ ]:
import pandas as pd

with open(OUTPUT_FILE, "r") as json_file:
    users_json = json.load(json_file)

rows = []
for user in users_json:
    row = {}
    id = user['user_id']
    row['user_id'] = id
    for col in user['values']:
        row[col] = user['values'][col][0]
    rows.append(row)

df = pd.DataFrame(rows)
df.to_csv('users_data.csv', index=False)

print("Archivo CSV guardado como 'users_data.csv'")

Archivo CSV guardado como 'usuarios.csv'


In [6]:
df = pd.read_csv('../../data/raw/users_data.csv')
df

,user_id,country,R,F,M
0,54601,25,1,37,49.785946
1,228633,25,278,2,66.850000
2,134234,25,241,3,42.280000
3,54600,25,24,6,45.614286
4,134230,25,83,1,17.950000
...,...,...,...,...,...
557001,262929,25,0,59,7.323467
557002,262931,25,5,100,13.240521
557003,262927,25,42,11,63.808333
557004,262930,25,143,6,32.613333


Tenemos ya el DataFrame de los usuarios